In [27]:
# -*- coding: utf-8 -*-
#
# Archivo: trading_bot.py
#
# Descripción: Un bot de trading inteligente que utiliza machine learning y feature engineering.
# Permite backtesting, medición de performance, y envío de señales.

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import ta  # Technical Analysis library
import requests
import json
import logging
import datetime
import yfinance as yf # New library for Yahoo Finance data
import joblib
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import os

# --- Configuración del Bot ---
# Configura el registro de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Clases del Bot de Trading ---

class FeatureEngineer:
    """
    Clase para la ingeniería de características a partir de datos de mercado.
    Incluye indicadores técnicos y otras características relevantes.
    """
    def __init__(self, df):
        if df is None:
            raise ValueError("El DataFrame proporcionado a FeatureEngineer es None.")
        self.df = df.copy()

    def add_technical_indicators(self):
        """
        Añade indicadores técnicos y otras características al DataFrame.
        """
        logging.info("Añadiendo indicadores técnicos avanzados...")
        
        # Indicadores de Tendencia
        self.df['SMA_50'] = ta.trend.sma_indicator(self.df['close'], window=50)
        self.df['EMA_20'] = ta.trend.ema_indicator(self.df['close'], window=20)
        
        # Indicadores de Momentum
        self.df['RSI'] = ta.momentum.rsi(self.df['close'], window=14)
        self.df['StochRSI'] = ta.momentum.stochrsi(self.df['close'], window=14)
        self.df['CCI'] = ta.trend.cci(self.df['high'], self.df['low'], self.df['close'])

        # Indicadores de Volatilidad
        self.df['Bollinger_high'] = ta.volatility.bollinger_hband(self.df['close'])
        self.df['Bollinger_low'] = ta.volatility.bollinger_lband(self.df['close'])
        self.df['ATR'] = ta.volatility.average_true_range(self.df['high'], self.df['low'], self.df['close'], window=14)
        
        # Indicadores de Volumen
        self.df['OBV'] = ta.volume.on_balance_volume(self.df['close'], self.df['volume'])
        self.df['ADI'] = ta.volume.acc_dist_index(self.df['high'], self.df['low'], self.df['close'], self.df['volume'])
        
        # Indicadores de Ichimoku Cloud
        self.df['ichimoku_conv'] = ta.trend.ichimoku_conversion_line(self.df['high'], self.df['low'])
        self.df['ichimoku_base'] = ta.trend.ichimoku_base_line(self.df['high'], self.df['low'])
        self.df['ichimoku_a'] = ta.trend.ichimoku_a(self.df['high'], self.df['low'])
        self.df['ichimoku_b'] = ta.trend.ichimoku_b(self.df['high'], self.df['low'])

        # Añadir indicadores adicionales para una justificación más precisa
        self.df['ADX'] = ta.trend.adx(self.df['high'], self.df['low'], self.df['close'])
        self.df['ADX_pos'] = ta.trend.adx_pos(self.df['high'], self.df['low'], self.df['close'])
        self.df['ADX_neg'] = ta.trend.adx_neg(self.df['high'], self.df['low'], self.df['close'])
        
        macd_obj = ta.trend.MACD(close=self.df['close'])
        self.df['MACD'] = macd_obj.macd()
        self.df['MACD_signal'] = macd_obj.macd_signal()
        self.df['MACD_diff'] = macd_obj.macd_diff()

        # Eliminación de filas con valores NaN
        self.df.dropna(inplace=True)
        return self.df

    def create_labels(self, window=10, threshold=0.01):
        """
        Crea etiquetas de tres clases para el entrenamiento del modelo.
        Etiqueta 2 (Compra): El precio sube un umbral en la ventana de tiempo.
        Etiqueta 0 (Venta): El precio baja un umbral en la ventana de tiempo.
        Etiqueta 1 (Mantener): De lo contrario.
        """
        logging.info("Creando etiquetas de clase...")
        future_price = self.df['close'].shift(-window)
        self.df['future_change'] = (future_price - self.df['close']) / self.df['close']
        
        self.df['label'] = 1  # Por defecto, la etiqueta es "Mantener"
        self.df.loc[self.df['future_change'] > threshold, 'label'] = 2  # "Compra"
        self.df.loc[self.df['future_change'] < -threshold, 'label'] = 0  # "Venta"
        
        self.df.dropna(inplace=True)
        return self.df

class ModelTrainer:
    """
    Clase para el entrenamiento del modelo de machine learning.
    Utiliza RandomForestClassifier con ajuste de hiperparámetros.
    """
    def __init__(self, data, features, label):
        self.data = data
        self.features = features
        self.label = label
        self.model = None

    def train_model(self):
        """
        Entrena el modelo de clasificación con ajuste de hiperparámetros.
        
        Justificación de ML: El modelo de machine learning (RandomForestClassifier)
        analiza patrones complejos en los indicadores técnicos. A diferencia de
        un algoritmo basado en reglas simples, este modelo puede identificar
        relaciones no lineales entre múltiples indicadores para tomar decisiones
        más sofisticadas y precisas, aprendiendo de datos históricos para
        predecir movimientos futuros del precio.
        """
        logging.info("Entrenando el modelo con ajuste de hiperparámetros...")
        X = self.data[self.features]
        y = self.data[self.label]
        
        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        
        # Definir el clasificador y los parámetros para el ajuste
        classifier = RandomForestClassifier(random_state=42)
        
        # Expandir la cuadrícula de búsqueda para un ajuste más completo
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        
        # Configurar la validación cruzada estratificada para manejar el desequilibrio de clases
        cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        # Crear la búsqueda de la cuadrícula
        # Entrenar la búsqueda de la cuadrícula en el conjunto de entrenamiento
        grid_search = GridSearchCV(classifier, param_grid, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        self.model = grid_search.best_estimator_
        
        logging.info(f"Mejores parámetros encontrados: {grid_search.best_params_}")
        
        # El modelo ya está entrenado, se devuelve para la evaluación
        return self.model, X_test, y_test

    def evaluate_model(self, model, X_test, y_test):
        """
        Evalúa el rendimiento del modelo.
        """
        logging.info("Evaluando el modelo...")
        y_pred = model.predict(X_test)
        print("Reporte de Clasificación:\n", classification_report(y_test, y_pred))
        print("Precisión (Accuracy):", accuracy_score(y_test, y_pred))

class Backtester:
    """
    Clase para el backtesting de la estrategia de trading.
    Simula trades basados en las predicciones del modelo.
    """
    def __init__(self, data, model):
        self.data = data.copy()
        self.model = model
        self.initial_capital = 1000.0
        self.capital = self.initial_capital
        self.position = 0.0
        self.transaction_cost = 0.001 # Costo de transacción del 0.1%

    def run_backtest(self):
        """
        Ejecuta la simulación de backtesting con una estrategia de tres señales.
        """
        logging.info("Ejecutando backtesting...")
        
        # Predecir señales para todo el conjunto de datos de backtesting
        X = self.data.dropna().drop(['label', 'future_change'], axis=1, errors='ignore')
        self.data['signal'] = self.model.predict(X)
        self.data['portfolio_value'] = np.nan
        
        # Simular trading
        for i in range(len(self.data)):
            # Lógica de entrada
            if self.data['signal'].iloc[i] == 2 and self.position == 0:
                # Comprar
                self.position = (self.capital * (1 - self.transaction_cost)) / self.data['close'].iloc[i]
                self.capital = 0
                logging.info(f"Compra en {self.data.index[i]}: {self.data['close'].iloc[i]}")

            # Lógica de salida
            elif self.data['signal'].iloc[i] == 0 and self.position > 0:
                # Vender
                self.capital = self.position * self.data['close'].iloc[i] * (1 - self.transaction_cost)
                self.position = 0
                logging.info(f"Venta en {self.data.index[i]}: {self.data['close'].iloc[i]}")

            # Actualizar el valor de la cartera
            if self.position > 0:
                self.data['portfolio_value'].iloc[i] = self.position * self.data['close'].iloc[i]
            else:
                self.data['portfolio_value'].iloc[i] = self.capital
        
        # Asegurarse de que el valor inicial del portafolio se establece
        self.data['portfolio_value'].iloc[0] = self.initial_capital

        # Rellenar los valores nulos del portafolio (para días sin transacciones)
        self.data['portfolio_value'] = self.data['portfolio_value'].ffill()

        return self.data

class PerformanceMetrics:
    """
    Clase para calcular métricas de rendimiento del backtesting.
    """
    def __init__(self, backtest_data):
        self.data = backtest_data

    def calculate_performance(self):
        """
        Calcula y muestra métricas de rendimiento avanzadas.
        """
        logging.info("Calculando métricas de rendimiento...")
        
        # Asegurarse de que el DataFrame de backtest tiene la columna 'portfolio_value'
        if 'portfolio_value' not in self.data.columns or self.data['portfolio_value'].isnull().all():
            logging.warning("No se pudo calcular el rendimiento del portafolio. Faltan datos de capital.")
            return

        # Retornos
        total_return = (self.data['portfolio_value'].iloc[-1] - self.data['portfolio_value'].iloc[0]) / self.data['portfolio_value'].iloc[0]
        print(f"Retorno Total: {total_return:.2%}")

        # Sharpe Ratio (ejemplo simple)
        returns = self.data['portfolio_value'].pct_change().dropna()
        if len(returns) > 0:
            sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252) # Asumiendo 252 días de trading
            print(f"Sharpe Ratio (Anualizado): {sharpe_ratio:.2f}")

        # Max Drawdown
        peak = self.data['portfolio_value'].expanding(min_periods=1).max()
        drawdown = (self.data['portfolio_value'] - peak) / peak
        max_drawdown = drawdown.min()
        print(f"Maximum Drawdown: {max_drawdown:.2%}")

def download_financial_data(ticker, asset_type, period=None, start=None, end=None, interval=None):
    """
    Función de ayuda para descargar datos financieros con un respaldo.
    Intenta yfinance primero, luego un segundo origen si falla.
    
    Nota: Para usar el respaldo de Stooq, podrías necesitar `pip install pandas_datareader`.
    """
    # Lista de formatos de ticker a intentar
    ticker_formats = [ticker]
    if asset_type == 'cripto':
        # Añade formatos comunes para criptomonedas
        ticker_formats.extend([f"{ticker}-USD", f"{ticker.upper()}-USD", f"{ticker.lower()}-USD"])
    # Para acciones, se podrían añadir sufijos de bolsa si fuera necesario, pero por ahora se mantiene simple

    # Lista de fuentes de datos a intentar
    data_sources = ['yfinance', 'stooq']

    for source in data_sources:
        for ticker_format in ticker_formats:
            try:
                if source == 'yfinance':
                    logging.info(f"Intentando descargar {ticker_format} de Yahoo Finance...")
                    df = yf.download(ticker_format, start=start, end=end, interval=interval, progress=False, auto_adjust=True)
                    if not df.empty:
                        df.columns = [col.lower() for col in df.columns]
                        logging.info(f"Éxito: Datos de {ticker_format} desde Yahoo Finance.")
                        return df
                elif source == 'stooq':
                    logging.info(f"Intentando descargar {ticker_format} de Stooq...")
                    # Stooq requiere la fecha de inicio/fin en un formato diferente, por lo que usamos pdr.
                    df = pdr.get_data_stooq(ticker_format, start=start, end=end)
                    if not df.empty:
                        df = df.sort_index(ascending=True)
                        df.columns = [col.lower() for col in df.columns]
                        logging.info(f"Éxito: Datos de {ticker_format} desde Stooq.")
                        return df
            except Exception as e:
                logging.warning(f"Fallo al intentar {ticker_format} desde {source}: {e}")
                continue # Continuar con el siguiente intento

    logging.error(f"Fallo en todas las fuentes para el ticker {ticker}. No se encontraron datos.")
    return None

def visualize_backtest(backtest_data, ticker):
    """
    Genera y muestra una gráfica de los resultados del backtesting.
    
    Justificación de las gráficas: Las gráficas son una herramienta vital
    para el análisis visual. Permiten superponer el rendimiento del portafolio,
    el precio del activo y los indicadores técnicos para verificar visualmente
    la efectividad de la estrategia y la coherencia de las señales del bot.
    """
    logging.info("Generando gráfica de resultados...")
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Trazar el valor del portafolio
    ax.plot(backtest_data.index, backtest_data['portfolio_value'], label='Valor del Portafolio', color='blue')
    
    # Trazar los precios de cierre
    ax.plot(backtest_data.index, backtest_data['close'], label='Precio de Cierre del Activo', color='gray', alpha=0.5)

    # Trazar el Ichimoku Cloud
    if 'ichimoku_conv' in backtest_data.columns and 'ichimoku_base' in backtest_data.columns:
        ax.plot(backtest_data.index, backtest_data['ichimoku_conv'], label='Ichimoku Conversion Line', color='orange')
        ax.plot(backtest_data.index, backtest_data['ichimoku_base'], label='Ichimoku Base Line', color='purple')
        if 'ichimoku_a' in backtest_data.columns and 'ichimoku_b' in backtest_data.columns:
            ax.fill_between(backtest_data.index, backtest_data['ichimoku_a'], backtest_data['ichimoku_b'], 
                            alpha=0.2, color='lightgreen', label='Ichimoku Cloud')

    # Identificar y marcar los puntos de compra y venta
    buy_signals = backtest_data[backtest_data['signal'] == 2]
    sell_signals = backtest_data[backtest_data['signal'] == 0]

    # Flechas de compra (hacia arriba en verde)
    for i in range(len(buy_signals)):
        ax.annotate(
            'Compra',
            xy=(buy_signals.index[i], buy_signals['close'].iloc[i]),
            xytext=(buy_signals.index[i], buy_signals['close'].iloc[i] * 0.95),
            arrowprops=dict(facecolor='green', shrink=0.05, width=2, headwidth=10),
            fontsize=8, color="green", ha="center"
        )

    # Flechas de venta (hacia abajo en rojo)
    for i in range(len(sell_signals)):
        ax.annotate(
            'Venta',
            xy=(sell_signals.index[i], sell_signals['close'].iloc[i]),
            xytext=(sell_signals.index[i], sell_signals['close'].iloc[i] * 1.05),
            arrowprops=dict(facecolor='red', shrink=0.05, width=2, headwidth=10),
            fontsize=8, color="red", ha="center"
        )

    # Añadir títulos y etiquetas
    ax.set_title(f'Resultados del Backtesting para {ticker}')
    ax.set_xlabel('Fecha')
    ax.set_ylabel('Valor ($)')
    ax.legend()
    ax.grid(True)
    plt.tight_layout()
    
    # Mostrar la gráfica
    plt.show()

def export_to_csv(df, filename):
    """
    Exporta el DataFrame a un archivo CSV.
    """
    try:
        df.to_csv(filename)
        logging.info(f"Datos exportados a {filename} con éxito.")
    except Exception as e:
        logging.error(f"Fallo al exportar los datos a CSV: {e}")

def summarize_backtest_results(df):
    """
    Genera un resumen del rendimiento del backtesting.
    """
    print("\n--- Resumen del Rendimiento del Backtesting ---")
    
    if df.empty:
        print("No hay datos de backtesting para resumir.")
        return
        
    start_price = df['close'].iloc[0]
    end_price = df['close'].iloc[-1]
    asset_return = (end_price - start_price) / start_price

    start_date = df.index[0].strftime('%Y-%m-%d')
    end_date = df.index[-1].strftime('%Y-%m-%d')
    
    print(f"Periodo de análisis: {start_date} a {end_date}")
    print(f"Retorno del activo durante el periodo: {asset_return:.2%}")
    
    total_trades = (df['signal'] == 2).sum()
    print(f"Número de operaciones de compra: {total_trades}")
    
    if 'portfolio_value' in df.columns and not df['portfolio_value'].isnull().all():
        total_return = (df['portfolio_value'].iloc[-1] - df['portfolio_value'].iloc[0]) / df['portfolio_value'].iloc[0]
        print(f"Retorno del portafolio (estrategia del bot): {total_return:.2%}")
    else:
        print("El retorno del portafolio no pudo ser calculado.")


def get_buy_justification(latest_data):
    """
    Genera una justificación textual para una señal de compra.
    """
    justification = []

    # Se comprueban los valores de los indicadores para una narrativa de compra.
    # Se añade un manejo de errores para las claves faltantes
    if 'RSI' in latest_data and latest_data['RSI'] < 30:
        justification.append("El RSI está por debajo de 30, lo que indica que el activo puede estar sobrevendido y podría recuperarse pronto.")
    if 'MACD' in latest_data and 'MACD_signal' in latest_data and latest_data['MACD'] > latest_data['MACD_signal']:
        justification.append("El MACD está por encima de su línea de señal, mostrando un impulso alcista creciente.")
    if 'ichimoku_conv' in latest_data and 'ichimoku_base' in latest_data and latest_data['ichimoku_conv'] > latest_data['ichimoku_base']:
        justification.append("La línea de conversión de Ichimoku ha cruzado por encima de la línea base, lo que sugiere un momentum alcista.")
    if 'ADX' in latest_data and 'ADX_pos' in latest_data and 'ADX_neg' in latest_data and latest_data['ADX'] > 25 and latest_data['ADX_pos'] > latest_data['ADX_neg']:
        justification.append("El ADX indica una tendencia fuerte, y la línea +DI está por encima de la -DI, confirmando la dirección alcista.")
    
    if not justification:
        return "El modelo de machine learning detectó una combinación favorable de múltiples indicadores que no se puede resumir con un solo patrón."
    
    return "Razones para la señal de compra:\n- " + "\n- ".join(justification)

def offline_training(ticker, asset_type):
    """
    Flujo de trabajo de entrenamiento offline (batch).
    """
    logging.info(f"Iniciando el entrenamiento offline para {ticker}...")
    try:
        # 1. Carga de datos
        end_date = datetime.date.today()
        start_date = end_date - datetime.timedelta(days=365) # 1 year of data
        df = download_financial_data(ticker, asset_type, start=start_date, end=end_date)
        if df is None:
            logging.info("Entrenamiento offline abortado debido a la falta de datos.")
            return None, None

        # 2. Ingeniería de Características
        fe = FeatureEngineer(df)
        df_fe = fe.add_technical_indicators()
        df_labeled = fe.create_labels()

        # 3. Entrenamiento del Modelo
        features = df_labeled.columns.drop(['label', 'future_change'], errors='ignore').tolist()
        mt = ModelTrainer(df_labeled, features, 'label')
        model, X_test, y_test = mt.train_model()
        mt.evaluate_model(model, X_test, y_test)

        # 4. Backtesting
        bt = Backtester(df_labeled, model)
        backtest_results = bt.run_backtest()
        
        # 5. Medición de Performance
        pm = PerformanceMetrics(backtest_results)
        pm.calculate_performance()
        
        # 6. Generar resumen y exportar datos
        summarize_backtest_results(backtest_results)
        export_to_csv(backtest_results, f'backtest_results_{ticker}.csv')

        # 7. Visualizar resultados
        visualize_backtest(backtest_results, ticker)

        logging.info("Entrenamiento offline finalizado.")
        return model, backtest_results
    except Exception as e:
        logging.error(f"Ocurrió un error en el entrenamiento offline: {e}")
        return None, None

def run_backtesting_with_existing_model(model, ticker, asset_type):
    """
    Flujo de trabajo para backtesting con un modelo ya entrenado.
    """
    logging.info(f"Iniciando el backtesting para {ticker} con modelo existente...")
    try:
        end_date = datetime.date.today()
        start_date = end_date - datetime.timedelta(days=365) # 1 year of data
        df = download_financial_data(ticker, asset_type, start=start_date, end=end_date)
        if df is None:
            logging.info("Backtesting abortado debido a la falta de datos.")
            return
        
        fe = FeatureEngineer(df)
        df_fe = fe.add_technical_indicators()
        df_labeled = fe.create_labels()

        bt = Backtester(df_labeled, model)
        backtest_results = bt.run_backtest()
        
        pm = PerformanceMetrics(backtest_results)
        pm.calculate_performance()

        summarize_backtest_results(backtest_results)
        export_to_csv(backtest_results, f'backtest_results_{ticker}.csv')

        visualize_backtest(backtest_results, ticker)
        logging.info("Backtesting con modelo existente finalizado.")

    except Exception as e:
        logging.error(f"Ocurrió un error en el backtesting: {e}")

def online_prediction(model, ticker, asset_type):
    """
    Flujo de trabajo de predicción online.
    """
    logging.info("Iniciando la predicción online...")
    try:
        # 1. Obtiene un periodo más largo de datos para asegurar el cálculo de indicadores
        new_data_df = download_financial_data(ticker, asset_type, period='60d', interval='1m')
        if new_data_df is None:
            logging.info("Predicción online abortada debido a la falta de datos.")
            return None
        
        # 2. Ingeniería de Características para los nuevos datos
        fe = FeatureEngineer(new_data_df)
        df_fe = fe.add_technical_indicators()

        # 3. Asegurarse de que las características coinciden
        model_features = model.feature_names_in_
        new_data_filtered = df_fe[model_features]
        
        # 4. Predicción en tiempo real
        if new_data_filtered.empty:
            logging.error("No hay suficientes datos para la predicción después de la ingeniería de características.")
            return None
            
        prediction = model.predict(new_data_filtered)

        # 5. Envío de Señal
        # Se toma la última predicción, que corresponde al dato más reciente
        latest_prediction = prediction[-1]
        
        if latest_prediction == 2:
            justification = get_buy_justification(new_data_filtered.iloc[-1])
            logging.info("Señal de compra detectada!")
            print(f"¡Señal de compra para {ticker}!")
            print(justification)
        elif latest_prediction == 0:
            logging.info("Señal de venta detectada!")
            print(f"¡Señal de venta para {ticker}!")
        else:
            logging.info("No hay señal de trading.")
        
        logging.info("Predicción online finalizada.")
        return latest_prediction
    except Exception as e:
        logging.error(f"Ocurrió un error en la predicción online: {e}")
        return None

def get_user_input():
    """
    Pregunta al usuario por el ticker y el tipo de activo.
    """
    asset_type = ''
    while asset_type.lower() not in ['cripto', 'accion']:
        asset_type = input("¿Deseas analizar una criptomoneda o una acción? (Escribe 'cripto' o 'accion'): ").lower()

    ticker_input = input("Introduce el ticker del activo (ej. BTC para cripto, AAPL para acción): ").upper()
    return ticker_input, asset_type

# --- Ejecución del Script ---

if __name__ == "__main__":
    
    TICKER, ASSET_TYPE = get_user_input()
    MODEL_PATH = f'trading_model_{TICKER}.pkl'

    model = None
    if os.path.exists(MODEL_PATH):
        try:
            model = joblib.load(MODEL_PATH)
            logging.info("Modelo cargado desde archivo.")
            
            # Opción para el usuario: backtesting o predicción
            action = ''
            while action.lower() not in ['backtest', 'prediccion']:
                action = input("¿Deseas 'backtest' con este modelo o hacer una 'prediccion' online? ").lower()

            if action == 'backtest':
                run_backtesting_with_existing_model(model, TICKER, ASSET_TYPE)
            else:
                online_prediction(model, TICKER, ASSET_TYPE)

        except (FileNotFoundError, ImportError) as e:
            logging.error(f"Error al cargar el modelo: {e}. Se intentará entrenar uno nuevo.")
            model, _ = offline_training(TICKER, ASSET_TYPE)
            if model:
                joblib.dump(model, MODEL_PATH)
                logging.info(f"Modelo guardado en {MODEL_PATH}")

    else:
        logging.warning("No se encontró el modelo, se procederá a entrenar uno nuevo.")
        model, _ = offline_training(TICKER, ASSET_TYPE)
        if model:
            joblib.dump(model, MODEL_PATH)
            logging.info(f"Modelo guardado en {MODEL_PATH}")

    if not model:
        logging.error("No se pudo cargar o entrenar un modelo. El script no continuará.")


¿Deseas analizar una criptomoneda o una acción? (Escribe 'cripto' o 'accion'):  cripto
Introduce el ticker del activo (ej. BTC para cripto, AAPL para acción):  ETH


2025-08-31 20:12:29,353 - INFO - Modelo cargado desde archivo.


¿Deseas 'backtest' con este modelo o hacer una 'prediccion' online?  prediccion


2025-08-31 20:12:33,809 - INFO - Iniciando la predicción online...
2025-08-31 20:12:33,809 - INFO - Intentando descargar ETH de Yahoo Finance...
2025-08-31 20:12:34,537 - ERROR - 
1 Failed download:
2025-08-31 20:12:34,538 - ERROR - ['ETH']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "1m data not available for startTime=1754010754 and endTime=1756689154. Only 8 days worth of 1m granularity data are allowed to be fetched per request.")')
2025-08-31 20:12:34,543 - INFO - Intentando descargar ETH-USD de Yahoo Finance...
2025-08-31 20:12:35,143 - ERROR - 
1 Failed download:
2025-08-31 20:12:35,144 - ERROR - ['ETH-USD']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "1m data not available for startTime=1754010754 and endTime=1756689154. Only 8 days worth of 1m granularity data are allowed to be fetched per request.")')
2025-08-31 20:12:35,150 - INFO - Intentando descargar ETH-USD de Yahoo Finance...
2

¡Señal de venta para ETH!
